In [1]:
from glob import glob

from collections import defaultdict
import json
import os

In [2]:
import os
import openai

openai.api_type = "azure"
openai.api_base = "https://husein-ai12-aiservices-1163379106.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''
engine = 'gpt-4'

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
files = glob('base-questions-loghat/*.json')

In [5]:
questions = []
for f in files:
    with open(f) as fopen:
        data = json.load(fopen)
    for q in data['questions']:
        questions.append(q['question'])
        
questions = sorted(list(set(questions)))
len(questions)

2027

In [6]:
questions

["'Rupe-rupenye' dalam Terengganu maksudnya apa?",
 "Adakah 'loghat baling' mengalami perubahan atau evolusi dalam beberapa dekad terakhir?",
 'Adakah Loghat Sabah mempunyai dialek-dialek yang berbeza mengikut daerah di Sabah?',
 'Adakah ada peraturan atau adat istiadat khusus yang mesti diikuti ketika berbicara dalam loghat Kelantan di majlis-majlis rasmi atau acara kebudayaan?',
 'Adakah ada perbedaan penggunaan loghat baling antara generasi muda dan tua?',
 'Adakah ada peribahasa atau ungkapan popular yang unik untuk loghat Perak?',
 'Adakah ada permainan atau hiburan tradisional di Baling yang menggunakan loghat setempat?',
 'Adakah ada permainan kata atau jenaka yang unik untuk loghat Johor?',
 'Adakah contoh frasa yang menggunakan loghat Sabah yang boleh menunjukkan identiti budaya Sabah?',
 'Adakah loghat Baling mempunyai kata-kata unik yang tidak ditemui dalam bahasa Melayu standard?',
 'Adakah loghat Baling mempunyai sebarang kata khas yang hanya digunakan semasa perayaan tert

In [7]:
questions[0]

"'Rupe-rupenye' dalam Terengganu maksudnya apa?"

In [9]:
!mkdir answer-random-questions-loghat

In [10]:
(len(questions) // 2) * 0, (len(questions) // 2) * 1

(0, 1013)

In [11]:
from tqdm import tqdm

for i in tqdm(range((len(questions) // 2) * 0, (len(questions) // 2) * 1, 1)):
    filename = f'answer-random-questions-loghat/{i}.json'
    if os.path.exists(filename):
        continue
        
    try:
        message_text = [
            {"role":"user","content": questions[i]},
        ]
        completion = openai.ChatCompletion.create(
          engine="gpt-4",
          messages = message_text,
          temperature=1.0,
          max_tokens=1024,
          top_p=0.95,
          frequency_penalty=0,
          presence_penalty=0,
          stop=None
        )
        splitted = completion.choices[0]['message']['content']
    except:
        splitted = None
    with open(filename, 'w') as fopen:
        json.dump(splitted, fopen)

100%|█████████████████████████████████████| 1013/1013 [5:51:25<00:00, 20.81s/it]


In [16]:
with open('loghat.jsonl', 'w') as fopen_l:

    for i, q in enumerate(questions):
        filename = f'answer-random-questions-loghat/{i}.json'
        if not os.path.exists(filename):
            continue
        
        try:
            with open(filename) as fopen:
                a = json.load(fopen).strip()
            
            d = {
                'question': q.strip(),
                'answer': a
            }

            fopen_l.write(f'{json.dumps(d)}\n')
            fopen_l.flush()  
        except:
            pass

In [17]:
!wc -l loghat.jsonl

2024 loghat.jsonl


In [18]:
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj='loghat.jsonl',
    path_in_repo='loghat.jsonl',
    repo_id='mesolitica/chatgpt4-malaysian-general-qa',
    repo_type='dataset',
)

loghat.jsonl:   0%|          | 0.00/2.89M [00:00<?, ?B/s]

'https://huggingface.co/datasets/mesolitica/chatgpt4-malaysian-general-qa/blob/main/loghat.jsonl'